In [7]:
import pandas as pd
pd.set_option('display.max_colwidth', None)
pd.options.mode.chained_assignment = None  # default='warn'

TRADUCTOR_SIGFE_SIGCOM = pd.read_excel('input\\asociacion_sigfe_sigcom.xlsx')
TRADUCTOR_SIGFE_SIGCOM['Cod. SIGFE limpio'] = TRADUCTOR_SIGFE_SIGCOM['Cod. SIGFE limpio'].str.replace('.', '', regex = False)
TRADUCTOR_SIGFE_SIGCOM = TRADUCTOR_SIGFE_SIGCOM.set_index('Cod. SIGFE limpio')
TRADUCTOR_SIGFE_SIGCOM = TRADUCTOR_SIGFE_SIGCOM.query('`Gastos SIGCOM` == "Gastos Generales"')

EXCEPCIONES_SIGFE = {
    '221299901601': 'FERNANDO BARAONA EN RRHH. ANEDIN EN GASTO GENERAL',
    '221299901602': 'Todo en RRHH',
    '221299900902': 'Cardiologia y Cardiocirugia en RRHH. UC Christus en gastos generales',
    '221299900201': 'M Meneses cargado en RRHH',
    '221299900202': 'J Andueza y Cardiocirugia en RRHH'
}

                   Cod. SIGCOM           Item en SIGCOM  \
Cod. SIGFE limpio                                         
2204006                    145   OTROS GASTOS GENERALES   
2204007001                 178   SERVICIO DE LAVANDERÍA   
2204014                    145   OTROS GASTOS GENERALES   
2204999                    145   OTROS GASTOS GENERALES   
2205001                     92      SERVICIO DE ENERGÍA   
...                        ...                      ...   
221100200602                76   CURSOS DE CAPACITACIÓN   
221299900202               177  SERVICIO DE LABORATORIO   
221299900901               145   OTROS GASTOS GENERALES   
221299900902               145   OTROS GASTOS GENERALES   
221299900903               145   OTROS GASTOS GENERALES   

                                                          Item SIGFE  \
Cod. SIGFE limpio                                                      
2204006              Fertilizantes, Insecticidas, Fungicidas y Otros   
2204007001      

In [4]:
class AnalizadorSIGCOM:
    def __init__(self):
        pass
    
    def correr_programa(self):
        estado_ej_presup, disponibilidad_devengo, formato_gg_sigcom = self.cargar_archivos_y_tratar_df()
        suma_gg_por_sigfe_total = self.obtener_suma_gg_total_por_sigfe(estado_ej_presup)
        suma_gg_por_sigfe_sin_rrhh, suma_rrhh = self.obtener_gastos_gg_y_rrhh(suma_gg_por_sigfe_total, disponibilidad_devengo)

        facturas_apiladas = self.obtener_detalle_facturas(suma_gg_por_sigfe_sin_rrhh, disponibilidad_devengo, suma_rrhh)

        suma_gg_por_sigcom_sin_rrhh = suma_gg_por_sigfe_sin_rrhh.groupby('Cod. SIGCOM', dropna = False).sum()

        suma_rrhh = suma_rrhh.groupby('Principal')['Monto Vigente'].sum().reset_index()
        suma_rrhh['Principal'] = suma_rrhh['Principal'].str.split(n = 1)
        suma_rrhh['Rut'] = suma_rrhh['Principal'].str[0]
        suma_rrhh['Nombre'] = suma_rrhh['Principal'].str[1]
        suma_rrhh = suma_rrhh[['Rut', 'Nombre', 'Monto Vigente']]


        formato_rellenado = self.rellenar_formato(suma_gg_por_sigcom_sin_rrhh, formato_gg_sigcom)

        self.guardar_archivos(suma_gg_por_sigfe_sin_rrhh, suma_rrhh, suma_gg_por_sigcom_sin_rrhh, formato_rellenado, facturas_apiladas)

    def cargar_archivos_y_tratar_df(self):
        estado_ej_presup = pd.read_excel('input\\SA_EstadoEjecucionPresupuestaria.xls', header = 6)
        estado_ej_presup['Item Presupuestario'] = estado_ej_presup['Concepto Presupuestario'].str.split().str[0]
        estado_ej_presup = estado_ej_presup.set_index('Item Presupuestario')
    
        disponibilidad_devengo = df_devengos = pd.read_excel('input\\SA_DisponibilidadDevengoPresupuestario.xls', header = 5)
        disponibilidad_devengo['Item Presupuestario'] = disponibilidad_devengo['Concepto Presupuestario'].str.split().str[0]
        disponibilidad_devengo = disponibilidad_devengo[['Titulo', 'Principal', 'Número Documento', 'Concepto Presupuestario', 'Item Presupuestario', 'Monto Vigente']]
        
        formato_gg_sigcom = pd.read_excel('input\\Formato 3_Gasto General 2022-09_2.xlsx')
        formato_gg_sigcom = formato_gg_sigcom.set_index('Unnamed: 0')

        return estado_ej_presup, disponibilidad_devengo, formato_gg_sigcom

    
    def obtener_suma_gg_total_por_sigfe(self, estado_ej_presup):
        unidas = pd.merge(estado_ej_presup, TRADUCTOR_SIGFE_SIGCOM, how = 'left', left_index = True, right_index = True)
    
        agrupados_por_sigfe = unidas.groupby(['Cod. SIGCOM', 'Item en SIGCOM', 'Item Presupuestario'], dropna = False)['Devengado'].sum().reset_index()
        agrupados_por_sigfe['Devengado_GG_consolidado'] = agrupados_por_sigfe['Devengado']

        agrupados_por_sigfe = agrupados_por_sigfe.rename(columns = {'Devengado': 'Devengado_EstadoEjecucionPresupuestaria',
                                                                    'Devengado_RRHH': 'Devengado_RRHH_DisponibilidadDevengoPresupuestario',
                                                                    'Devengado_GG': 'Devengado_GG_DisponibilidadDevengoPresupuestario'})

        return agrupados_por_sigfe
    
    def obtener_detalle_facturas(self, suma_gg_por_sigfe_sin_rrhh, disponibilidad_devengo, suma_rrhh):
        '''
        Esta función permite obtener el detalle de cada factura involucrada en el gasto general del item SIGCOM.
        Para esto, toma los items presupuestarios involucrados en el gasto general y busca las facturas en la disponibilidad de devengo.

        En primer lugar, es necesario filtrar la suma_gg_por_sigfe_sin_rrhh, ya que contiene items SIGFE sin un item SIGCOM. 
        '''
        facturas_apiladas = pd.DataFrame()
        suma_gg_por_sigfe_sin_rrhh = suma_gg_por_sigfe_sin_rrhh.dropna(subset = 'Cod. SIGCOM')

        for item_presupuestario, codigo_sigcom in zip(suma_gg_por_sigfe_sin_rrhh['Item Presupuestario'], suma_gg_por_sigfe_sin_rrhh['Cod. SIGCOM']):
            query_facturas = disponibilidad_devengo.query('`Item Presupuestario` == @item_presupuestario')
            query_facturas['Cod. SIGCOM'] = codigo_sigcom

            facturas_apiladas = pd.concat([facturas_apiladas, query_facturas])

        # Ahora, hay que quitar las facturas que van a rrhh, y listo
        facturas_apiladas = pd.concat([facturas_apiladas, suma_rrhh])
        facturas_apiladas = facturas_apiladas.drop_duplicates(subset = ['Principal', 'Número Documento', 'Monto Vigente'], keep = False)

        # facturas_con_oc = facturas_apiladas[facturas_apiladas['Titulo'].str.contains('/')]
        # facturas_sin_oc = facturas_apiladas[~facturas_apiladas['Titulo'].str.contains('/')]

        # facturas_con_oc['folio_oc'] = facturas_con_oc['Titulo'].str.split('/').str[3]
        
        return facturas_apiladas

    
    def obtener_gastos_gg_y_rrhh(self, suma_gg_por_sigfe, disponibilidad_devengo):
        df_rrhh_total = pd.DataFrame()
        suma_gg_por_sigfe_sin_rrhh = suma_gg_por_sigfe.copy()

        for concepto_pres_excepcion in EXCEPCIONES_SIGFE.keys():
            query_excepcion = disponibilidad_devengo.query('`Item Presupuestario` == @concepto_pres_excepcion')

            if concepto_pres_excepcion == '221299901601':
                mask_a_rrhh = query_excepcion['Principal'].str.contains('BARAONA')
            
            elif concepto_pres_excepcion == '221299901602':
                mask_a_rrhh = query_excepcion['Principal'].notna()
            
            elif concepto_pres_excepcion == '221299900902':
                mask_a_rrhh = query_excepcion['Principal'].str.contains('CARDIOLOGIA') | (query_excepcion['Principal'].str.contains('CARDIOCIRUGIA'))

            elif concepto_pres_excepcion == '221299900201':
                mask_a_rrhh = query_excepcion['Principal'].str.contains('MANUEL MENESES')
                
            elif concepto_pres_excepcion == '221299900202':
                mask_a_rrhh = query_excepcion['Principal'].str.contains('ANDUEZA') | query_excepcion['Principal'].str.contains('CARDIOCIRUGIA')
            
            df_a_rrhh = query_excepcion[mask_a_rrhh]
            df_rrhh_total = pd.concat([df_rrhh_total, df_a_rrhh])

            df_a_gg = query_excepcion[~mask_a_rrhh]

            valor_a_rrhh = df_a_rrhh['Monto Vigente'].sum()
            valor_a_gg = df_a_gg['Monto Vigente'].sum()

            mask_excepcion = (suma_gg_por_sigfe_sin_rrhh['Item Presupuestario'] == concepto_pres_excepcion)

            suma_gg_por_sigfe_sin_rrhh.loc[mask_excepcion, 'Devengado_RRHH'] = valor_a_rrhh 
            suma_gg_por_sigfe_sin_rrhh.loc[mask_excepcion, 'Devengado_GG'] = valor_a_gg
            suma_gg_por_sigfe_sin_rrhh.loc[mask_excepcion, 'Devengado_GG_consolidado'] = valor_a_gg

        return suma_gg_por_sigfe_sin_rrhh, df_rrhh_total
    
    def rellenar_formato(self, suma_gg_por_sigcom_sin_rrhh, formato_sigcom_gg):
        columnas_antiguas = formato_sigcom_gg.columns
        formato_sigcom_gg.columns = formato_sigcom_gg.columns.str.split('-').str[0].astype(float)

        for codigo_gasto in suma_gg_por_sigcom_sin_rrhh.index:
            if codigo_gasto in formato_sigcom_gg.columns:
                valor_a_ingresar = suma_gg_por_sigcom_sin_rrhh.loc[codigo_gasto, 'Devengado_GG_consolidado']
                formato_sigcom_gg.loc['Valor General', codigo_gasto] = valor_a_ingresar

        formato_sigcom_gg.columns = columnas_antiguas
        formato_sigcom_gg = formato_sigcom_gg.reset_index()
        
        return formato_sigcom_gg
    
    def guardar_archivos(self, suma_gg_por_sigfe_sin_rrhh, suma_rrhh, suma_gg_por_sigcom_sin_rrhh, formato_rellenado, facturas_apiladas):
        suma_gg_por_sigfe_sin_rrhh = suma_gg_por_sigfe_sin_rrhh.reset_index()
        suma_gg_por_sigcom_sin_rrhh = suma_gg_por_sigcom_sin_rrhh.reset_index()
        suma_rrhh = suma_rrhh.reset_index()
        formato_rellenado = formato_rellenado.reset_index(drop = True)
        facturas_apiladas = facturas_apiladas.reset_index()

        with pd.ExcelWriter('output.xlsx') as writer:
            formato_rellenado.to_excel(writer, sheet_name = 'formato_listo', index = False)
            suma_gg_por_sigfe_sin_rrhh.to_excel(writer, sheet_name = 'suma_gg_por_sigfe', index = False)
            suma_gg_por_sigcom_sin_rrhh.to_excel(writer, sheet_name = 'suma_gg_por_sigcom', index = False)
            suma_rrhh.to_excel(writer, sheet_name = 'suma_rrhh', index = False)
            facturas_apiladas.to_excel(writer, sheet_name = 'facturas_apiladas', index = False)


In [5]:
objeto = AnalizadorSIGCOM()
objeto.correr_programa()

In [6]:
TRADUCTOR_SIGFE_SIGCOM

,Cod. SIGCOM,Item en SIGCOM,Item SIGFE,Cod. SIGFE,Gastos SIGCOM
Cod. SIGFE limpio,,,,,
2201001,46,VÍVERES,BIENES Y SERVICIOS DE CONSUMO: Para Personas,22.01.001 y sub asignaciones,Insumos
2201002,46,VÍVERES,Para Animales,22.01.002,Insumos
2202001,43,"PRODUCTOS TEXTILES, VESTUARIO Y CALZADO",Textiles y Acabados Textiles,22.02.001,Insumos
2202002,43,"PRODUCTOS TEXTILES, VESTUARIO Y CALZADO",Vest Y Acces P Diversas Para El Personal,22.02.002 y sub asignaciones,Insumos
2202003,43,"PRODUCTOS TEXTILES, VESTUARIO Y CALZADO",Calzado,22.02.003,Insumos
...,...,...,...,...,...
221100200602,76,CURSOS DE CAPACITACIÓN,Convenios Con Universidades Ley 19664,221100200602,Gastos Generales
221299900202,177,SERVICIO DE LABORATORIO,Compra de Procedimientos,221299900202,Gastos Generales
221299900901,145,OTROS GASTOS GENERALES,Convenios Dfl 36 Centralizados,221299900901,Gastos Generales
